In [1]:
import sys
import tensorflow as tf
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)
SEED=27

tf.random.set_seed(SEED)

print('Python version:', sys.version)
print('Tensorflow version:', tf.__version__)

Python version: 3.8.9 (default, Apr  3 2021, 01:50:09) 
[Clang 12.0.0 (clang-1200.0.32.29)]
Tensorflow version: 2.4.1


My local imports

In [2]:
cd ..

/Users/n.barsukov/PycharmProjects/toxic-comments-detector


In [3]:
from textPreprocessing import preprocess_text
from wordEmbeddingsLayers import wiki40_russian_embedding_layer

# Data preparation

In [4]:
orig_toxic_comments_df = pd.read_csv('data/toxicCommentsOriginalDF.csv')
orig_toxic_comments_df.head(3)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0


Preprocess texts

In [5]:
df = orig_toxic_comments_df.copy()

df['comment'] = list(map(preprocess_text, df['comment']))

df.head(3)

,comment,toxic
0,верблюд то за что дебил бл,1.0
1,хохол это отдушина затюканый россиянин мол вон а у хохлов ещё плохой если бы хохлов не быть кисель они бы придумать,1.0
2,собака собачий смерть,1.0


Train / test dataframe split

In [6]:
from sklearn.model_selection import train_test_split

y = df.pop('toxic')
X = np.array(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Build RNN model

In [7]:
from tensorflow.keras import layers, losses

model = tf.keras.Sequential([
    layers.Input(shape=[], dtype=tf.string),
    wiki40_russian_embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    layers.Dropout(rate=0.2, seed=SEED),
    tf.keras.layers.Dense(64, activation='relu'),
    layers.Dense(1),
    layers.Activation('sigmoid') # то есть в конце мы выдаем уже вероятности
])

model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), # from_logits=False, потому что у нас есть финальный 
    optimizer='adam',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)

In [8]:
epochs = 10
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    batch_size=2**7,
    validation_split=0.2, # сколько от тестовой выборки отрезать под валидационную
)

Epoch 1/10
73/73 [==============================] - 64s 844ms/step - loss: 0.5389 - binary_accuracy: 0.7179 - recall: 0.3631 - precision: 0.6432 - val_loss: 0.3369 - val_binary_accuracy: 0.8539 - val_recall: 0.7174 - val_precision: 0.8212
Epoch 2/10
73/73 [==============================] - 60s 828ms/step - loss: 0.2945 - binary_accuracy: 0.8822 - recall: 0.8057 - precision: 0.8410 - val_loss: 0.2915 - val_binary_accuracy: 0.8786 - val_recall: 0.7930 - val_precision: 0.8342
Epoch 3/10
73/73 [==============================] - 60s 818ms/step - loss: 0.2206 - binary_accuracy: 0.9145 - recall: 0.8622 - precision: 0.8796 - val_loss: 0.4626 - val_binary_accuracy: 0.7984 - val_recall: 0.9388 - val_precision: 0.6330
Epoch 4/10
73/73 [==============================] - 59s 804ms/step - loss: 0.1945 - binary_accuracy: 0.9275 - recall: 0.9012 - precision: 0.8892 - val_loss: 0.3117 - val_binary_accuracy: 0.8851 - val_recall: 0.7643 - val_precision: 0.8748
Epoch 5/10
73/73 [==========================

# Model evaluation

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

loss, accuracy, recall, precision = model.evaluate(x=X_test, y=y_test)
y_test_pred = (model.predict(X_test).flatten() > 0.5) * 1

print("\nLoss: ", loss)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("Precision: ", precision, '\n')

print(classification_report(y_true=y_test, y_pred=y_test_pred, target_names=['non_toxic', 'toxic']))

print('Confusion matrix:')
print(confusion_matrix(y_true=y_test, y_pred=y_test_pred))

91/91 [==============================] - 6s 64ms/step - loss: 0.4114 - binary_accuracy: 0.8998 - recall: 0.8103 - precision: 0.8774

Loss:  0.4113820195198059
Accuracy:  0.8997572064399719
Recall:  0.8102725148200989
Precision:  0.8774120211601257 

              precision    recall  f1-score   support

   non_toxic       0.91      0.94      0.93      1929
       toxic       0.88      0.81      0.84       954

    accuracy                           0.90      2883
   macro avg       0.89      0.88      0.88      2883
weighted avg       0.90      0.90      0.90      2883

Confusion matrix:
[[1821  108]
 [ 181  773]]
